# Download the PaleoCAR data files

### Get list of NetCDF files in dataset directory on NOAA web site

In [31]:
import ipywidgets as widgets
import os.path
import pandas as pd
import requests
import osr
import numpy as np
from osgeo import gdal

### Define url of directory at NOAA containing the data files to download

In [32]:
dataset_url = 'http://www1.ncdc.noaa.gov/pub/data/paleo/treering/reconstructions/northamerica/usa/bocinsky2016/'
default_destination_dir = '/projects/skope/original/paleocar/'

### Get the names of files corresponding to each of the two data set variables

In [33]:
# load first html table at url into a dataframe
table = pd.read_html(dataset_url)[0]

# discard table rows with nulls in column 1
valid_rows = table.dropna(subset=[1])

# get column of file names
file_names = valid_rows[1]

# get names of files corresponding to the GDD and PPT variables
gdd_file_names = list(file_names[file_names.str.contains('GDD.nc4')])
ppt_file_names = list(file_names[file_names.str.contains('PPT.nc4')])

gdd_total_file_count = len(gdd_file_names)
ppt_total_file_count = len(ppt_file_names)

# make sure there are 155 files for each variable
assert gdd_total_file_count == 155
assert ppt_total_file_count == 155

### Specify the number of files to download for each variable

In [34]:
download_to_directory = widgets.Text(
    description = "Download to", value=default_destination_dir)

gdd_selected_file_count = widgets.IntSlider(
    description = "GDD files", 
    min         = 1, 
    max         = gdd_total_file_count, 
    value       = gdd_total_file_count
)

ppt_selected_file_count = widgets.IntSlider(
    description = "PPT files", 
    min         = 1, 
    max         = ppt_total_file_count, 
    value       = ppt_total_file_count
)

download_status = widgets.Button(
    description = 'Download not started',
    disabled    = True
)

display(
    download_to_directory,
    gdd_selected_file_count, 
    ppt_selected_file_count, 
    download_status
)

Text(value='/projects/skope/original/paleocar/', description='Download to')

IntSlider(value=155, description='GDD files', max=155, min=1)

IntSlider(value=155, description='PPT files', max=155, min=1)

Button(description='Download not started', disabled=True, style=ButtonStyle())

### Download the selected number of files for each variable of the dataset

In [35]:
destination_dir = download_to_directory.value

def download_file(source_url, destination_path):
    response = requests.get(source_url)
    assert response.status_code == 200
    with open(destination_path, 'wb') as f:  
        f.write(response.content)

def download_file_if_needed(base_url, file_name, destination_dir):
    source_url = base_url + file_name
    destination_path = destination_dir + '/' + file_name
    if os.path.exists(destination_path):
        print(file_name, "already downloaded")
    else:
        download_file(source_url, destination_path)
        print(file_name, "finished downloading")

def download_files(file_list, destination_dir):
    for i, file_name in enumerate(file_list):
        download_file_if_needed(dataset_url, file_name, destination_dir)
    print("\nTotal of %d files downloaded\n" % (i+1))

download_status.description = 'Downloading files'
download_files(gdd_file_names[:gdd_selected_file_count.value], destination_dir)
download_files(ppt_file_names[:ppt_selected_file_count.value], destination_dir)
download_status.description = 'Downloads complete'


103W31N_GDD.nc4 already downloaded
103W32N_GDD.nc4 already downloaded
103W33N_GDD.nc4 already downloaded
103W34N_GDD.nc4 already downloaded
103W35N_GDD.nc4 already downloaded
103W36N_GDD.nc4 already downloaded
103W37N_GDD.nc4 already downloaded
103W38N_GDD.nc4 already downloaded
103W39N_GDD.nc4 already downloaded
103W40N_GDD.nc4 already downloaded
103W41N_GDD.nc4 already downloaded
103W42N_GDD.nc4 already downloaded
104W31N_GDD.nc4 already downloaded
104W32N_GDD.nc4 already downloaded
104W33N_GDD.nc4 already downloaded
104W34N_GDD.nc4 already downloaded
104W35N_GDD.nc4 already downloaded
104W36N_GDD.nc4 already downloaded
104W37N_GDD.nc4 already downloaded
104W38N_GDD.nc4 already downloaded
104W39N_GDD.nc4 already downloaded
104W40N_GDD.nc4 already downloaded
104W41N_GDD.nc4 already downloaded
104W42N_GDD.nc4 already downloaded
105W31N_GDD.nc4 already downloaded
105W32N_GDD.nc4 already downloaded
105W33N_GDD.nc4 already downloaded
105W34N_GDD.nc4 already downloaded
105W35N_GDD.nc4 alre

# Display properties of the first downloaded data file

### Load the first GDD data file into a GDAL dataset object

In [138]:
# get path to first GDD data file
path_to_first_gdd_file = destination_dir + "/" + gdd_file_names[0]

# open the first GDD data file
dataset = gdal.Open(path_to_first_gdd_file)

# confirm that a Dataset object was returned
assert str((type(dataset))) == "<class 'osgeo.gdal.Dataset'>"

### Load and display elements of the raster-space to coordinate-space transform

In [147]:
# get the transform from the data file
geo_transform = dataset.GetGeoTransform()

# confirm that this indeed is a North-Up transformation (ignored elements of transform are both zero)
assert geo_transform[2] == 0
assert geo_transform[4] == 0

# extract the origin and pixel dimensions from the transform
origin_x      = geo_transform[0]
origin_y      = geo_transform[3]
pixel_size_x  = abs(geo_transform[1])
pixel_size_y  = abs(geo_transform[5])

# summarize the transform
print("Origin:          %11.6f,%11.6f" % (origin_x, origin_y))
print("Pixel size:      %11.6f,%11.6f" % (pixel_size_x, pixel_size_y))
print("Pixels/degree:   %11.6f,%11.6f" % (1.0/pixel_size_x, 1.0/pixel_size_y))
print("Top-left corner: %11.6f,%11.6f" % (origin_x - pixel_size_x/2, origin_y - pixel_size_y/2))

Origin:          -102.995833,  31.995833
Pixel size:         0.008333,   0.008333
Pixels/degree:    120.000000, 120.000000
Top-left corner: -103.000000,  31.991667


### Confirm that dimensions of data set is 120 x 120 pixels and 2000 bands

In [148]:
# extract the dimensions from the data file 
extent_in_pixels_x = dataset.RasterXSize
extent_in_pixels_y = dataset.RasterYSize
band_count         = dataset.RasterCount

# confirm the dimensions are those expected
assert extent_in_pixels_x == 120
assert extent_in_pixels_y == 120
assert band_count         == 2000

# summarize the data file dimensions
print('Dimensions: %d x %d pixels and %d bands' % (
    extent_in_pixels_x, 
    extent_in_pixels_y, 
    band_count
    )
)

Dimensions: 120 x 120 pixels and 2000 bands


### Note that the GetProjection() method returns an empty string for this data file

In [80]:
print("Value returned from GetProjection() method on dataset: '%s'" % (
    dataset.GetProjection())
)

assert str(type(dataset.GetProjection())) == "<class 'str'>"
assert dataset.GetProjection()            == ''

Value returned from GetProjection() method on dataset: ''
